In [ ]:
import pandas as pd
import os
from collections import defaultdict
import networkx as nx
from itertools import combinations
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
myDf = pd.read_csv('/Users/mraskin/Downloads/msk_met_2021_clinical_data.tsv', sep='\t')

In [ ]:
myDf.columns

In [ ]:
myDf.columns = ['_'.join(x.lower().split(' ')) for x in myDf.columns]

In [ ]:
myDf.columns

In [ ]:
myDf.subtype_abbreviation.unique()

In [ ]:
myDf.columns

In [ ]:
myDf.cancer_type.value_counts()

In [ ]:
myDf = myDf[myDf.cancer_type.isin(myDf.cancer_type.value_counts().iloc[0:10].index)]

In [ ]:
myDf

In [ ]:
met_cols = [col for col in myDf.columns if col.startswith("distant_mets:_")]

In [ ]:
len(met_cols)

In [ ]:
myDf[met_cols] = (myDf[met_cols] == "Yes").astype(int)

In [ ]:
new_met_cols = [x.split(':_')[-1] for x in met_cols]
myDf[new_met_cols] = myDf[met_cols]

In [ ]:
def get_transition_probs(num_metastasis):
    df_melted = myDf[(myDf.sample_type == "Primary") & (myDf.met_count == num_metastasis)].melt(
        id_vars=["patient_id", "cancer_type", "metastatic_site"],
        value_vars=new_met_cols,
        var_name="metastatic_site_dist",
        value_name="Present"
    )

    df_melted = df_melted[df_melted["Present"] == 1]

    contingency = pd.crosstab(index=[df_melted['cancer_type']], columns=df_melted["metastatic_site_dist"])
    # contingency
    transition_probs_3 = contingency.div(contingency.sum(axis=1), axis=0)
    myCancerTypes = contingency.T.sum()
    if transition_probs_3.shape[1] == 21:
        norm = transition_probs_3.values
        norm_T = norm.T / np.sum(norm, axis=1)
        return norm_T, myCancerTypes
    return None, myCancerTypes

In [ ]:
transition_probs_1, _ = get_transition_probs(1)

In [ ]:
transition_probs_2, _ = get_transition_probs(2)

In [3040]:
def get_expected_prob_dist(num_metastasis, j):
    P_direct = transition_probs_1.T
    C, residuals, rank, s = np.linalg.lstsq((1-j)*P_direct, transition_probs_2.T - j * P_direct, rcond=None)
    C = C.clip(0.1, 0.9)
    C = C.T / C.sum(axis=1)
    P_n = P_direct
    for i in range(2, num_metastasis+1):
        C_new = np.linalg.matrix_power(C, i-1)
        P_branching = C_new @ P_n.T
        P_n = j * P_n + (1-j) * P_branching.T
    return P_n.T, P_direct

In [3041]:
import numpy as np
from scipy.stats import chisquare

chi2_results = {}
j = 2
for j in range(0, 10):
    for c in range(10):
        for i in range(3, 12):
            P_n_T, P_direct = get_expected_prob_dist(i, j * 0.1)
            norm_T, myCancerTypes = get_transition_probs(i)
            if norm_T is not None:
                observed_counts = norm_T * myCancerTypes.to_numpy()
                expected_counts = P_n_T* myCancerTypes.to_numpy()
                    
                obs = observed_counts[:, c].flatten()
                exp = expected_counts[:, c].flatten()
                chi2, pval = chisquare(obs+1e-10, f_exp=exp+1e-10)
                if pval > 0.95:
                    print({
                        (round(j * 0.1,2), i, myCancerTypes.reset_index().iloc[c].cancer_type, myCancerTypes.iloc[c]):
                        (round(chi2, 2), round(pval, 2))
                    })
            else:
                print(f'Not Computing {i}')

{(0.0, 11, 'Hepatobiliary Cancer', 8): (9.84, 0.97)}
{(0.1, 11, 'Hepatobiliary Cancer', 8): (9.85, 0.97)}
{(0.2, 11, 'Hepatobiliary Cancer', 8): (9.87, 0.97)}
{(0.3, 11, 'Hepatobiliary Cancer', 8): (9.94, 0.97)}
{(0.4, 11, 'Hepatobiliary Cancer', 8): (10.07, 0.97)}
{(0.5, 11, 'Hepatobiliary Cancer', 8): (10.22, 0.96)}
{(0.6, 11, 'Hepatobiliary Cancer', 8): (10.27, 0.96)}
{(0.7, 11, 'Hepatobiliary Cancer', 8): (10.23, 0.96)}
{(0.8, 11, 'Hepatobiliary Cancer', 8): (10.32, 0.96)}


In [ ]:
import numpy as np
from scipy.stats import chisquare

best_results = {}  # keys: (i, c), values: (best_j, chi2, pval)

for j in range(10):
    j_val = j * 0.1
    for c in range(10):
        for i in range(3, 12):
            P_n_T, P_direct = get_expected_prob_dist(i, j_val)
            norm_T, myCancerTypes = get_transition_probs(i)
            
            if norm_T is not None:
                observed_counts = norm_T * myCancerTypes.to_numpy()
                expected_counts = P_n_T * myCancerTypes.to_numpy()

                obs = observed_counts[:, c].flatten()
                exp = expected_counts[:, c].flatten()

                chi2, pval = chisquare(obs + 1e-10, f_exp=exp + 1e-10)

                key = (i, myCancerTypes.index[c])
                if key not in best_results or pval > best_results[key][2]:
                    best_results[key] = (j_val, chi2, pval)
            else:
                pass

for (i, c), (best_j, chi2, pval) in best_results.items():
    print(f"i={i}, cancer_index={c}, best_j={best_j:.1f}, chi2={chi2:.2f}, pval={pval:.3f}")

i=3, cancer_index=Bladder Cancer, best_j=0.6, chi2=115.46, pval=0.000
i=4, cancer_index=Bladder Cancer, best_j=0.6, chi2=169.74, pval=0.000
i=5, cancer_index=Bladder Cancer, best_j=0.6, chi2=97.66, pval=0.000
i=6, cancer_index=Bladder Cancer, best_j=0.6, chi2=115.57, pval=0.000
i=7, cancer_index=Bladder Cancer, best_j=0.7, chi2=57.96, pval=0.000
i=8, cancer_index=Bladder Cancer, best_j=0.7, chi2=69.23, pval=0.000
i=9, cancer_index=Bladder Cancer, best_j=0.0, chi2=31.92, pval=0.044
i=10, cancer_index=Bladder Cancer, best_j=0.1, chi2=66.14, pval=0.000
i=11, cancer_index=Bladder Cancer, best_j=0.2, chi2=14.01, pval=0.830
i=3, cancer_index=Breast Cancer, best_j=0.6, chi2=197.09, pval=0.000
i=4, cancer_index=Breast Cancer, best_j=0.5, chi2=180.93, pval=0.000
i=5, cancer_index=Breast Cancer, best_j=0.0, chi2=208.92, pval=0.000
i=6, cancer_index=Breast Cancer, best_j=0.0, chi2=170.92, pval=0.000
i=7, cancer_index=Breast Cancer, best_j=0.7, chi2=141.27, pval=0.000
i=8, cancer_index=Breast Canc

In [3037]:
from scipy.stats import chisquare
import numpy as np

pval_sums = {}
for j in range(0, 10):
    for i in range(3, 12):
        P_n_T, P_direct = get_expected_prob_dist(i, j * 0.1)
        norm_T, myCancerTypes = get_transition_probs(i)
        if norm_T is not None:
            observed_counts = norm_T * myCancerTypes.to_numpy()
            expected_counts = P_n_T * myCancerTypes.to_numpy()
            for c in range(10):
                obs = observed_counts[:, c].flatten()
                exp = expected_counts[:, c].flatten()

                chi2, pval = chisquare(obs+1e-10, f_exp=exp+1e-10)

                cancer_type = myCancerTypes.reset_index().iloc[c].cancer_type

                if cancer_type not in pval_sums:
                    pval_sums[cancer_type] = {}
                if j not in pval_sums[cancer_type]:
                    pval_sums[cancer_type][j] = 0
                pval_sums[cancer_type][j] += pval
        else:
            print(f'Not Computing {i}')

for cancer_type, j_dict in pval_sums.items():
    best_j = max(j_dict, key=j_dict.get)
    best_pval_sum = j_dict[best_j]
    print(f"Cancer type: {cancer_type}, best j: {best_j * 0.1:.1f}, average of p-values: {best_pval_sum / len(range(3, 12)):.4f}")

Cancer type: Bladder Cancer, best j: 0.7, average of p-values: 0.0693
Cancer type: Breast Cancer, best j: 0.8, average of p-values: 0.0000
Cancer type: Colorectal Cancer, best j: 0.7, average of p-values: 0.0000
Cancer type: Endometrial Cancer, best j: 0.8, average of p-values: 0.0005
Cancer type: Hepatobiliary Cancer, best j: 0.8, average of p-values: 0.1050
Cancer type: Melanoma, best j: 0.7, average of p-values: 0.0934
Cancer type: Non-Small Cell Lung Cancer, best j: 0.9, average of p-values: 0.0000
Cancer type: Ovarian Cancer, best j: 0.9, average of p-values: 0.0343
Cancer type: Pancreatic Cancer, best j: 0.8, average of p-values: 0.0001
Cancer type: Prostate Cancer, best j: 0.5, average of p-values: 0.0355


In [3008]:
from scipy.stats import chisquare, chi2

cancer_j_stats = {}
for j in range(0, 10):
    for i in range(3, 12):
        P_n_T, P_direct = get_expected_prob_dist(i, j * 0.1)
        norm_T, myCancerTypes = get_transition_probs(i)
        if norm_T is not None:
            observed_counts = norm_T * myCancerTypes.to_numpy()
            expected_counts = P_n_T * myCancerTypes.to_numpy()
            for c in range(10):
                obs = observed_counts[:, c].flatten()
                exp = expected_counts[:, c].flatten()

                obs_adj = obs+1e-10
                exp_adj = exp+1e-10

                chi2_stat, pval = chisquare(obs_adj, f_exp=exp_adj)
                df = len(obs_adj)

                cancer_type = myCancerTypes.reset_index().iloc[c].cancer_type

                if cancer_type not in cancer_j_stats:
                    cancer_j_stats[cancer_type] = {}
                if j not in cancer_j_stats[cancer_type]:
                    cancer_j_stats[cancer_type][j] = [0, 0]

                cancer_j_stats[cancer_type][j][0] += chi2_stat
                cancer_j_stats[cancer_type][j][1] += df-1
        else:
            print(f'Not Computing {i}')

best_j_per_cancer = {}
for cancer_type, j_dict in cancer_j_stats.items():
    best_pval = -1
    best_j = None
    for j, (sum_chi2, sum_df) in j_dict.items():
        global_pval = chi2.sf(sum_chi2, sum_df)
        if global_pval > best_pval:
            best_pval = global_pval
            best_j = j
    best_j_per_cancer[cancer_type] = (best_j * 0.1, best_pval)

for cancer_type, (best_j_val, pval) in best_j_per_cancer.items():
    print(f"Cancer type: {cancer_type}, best j: {best_j_val:.2f}, global p-value: {pval:.4g}")

Cancer type: Bladder Cancer, best j: 0.60, global p-value: 2.067e-70
Cancer type: Breast Cancer, best j: 0.60, global p-value: 4.159e-203
Cancer type: Colorectal Cancer, best j: 0.00, global p-value: 0
Cancer type: Endometrial Cancer, best j: 0.80, global p-value: 1.652e-69
Cancer type: Hepatobiliary Cancer, best j: 0.80, global p-value: 2.392e-70
Cancer type: Melanoma, best j: 0.60, global p-value: 1.441e-09
Cancer type: Non-Small Cell Lung Cancer, best j: 0.00, global p-value: 0
Cancer type: Ovarian Cancer, best j: 0.80, global p-value: 3.672e-55
Cancer type: Pancreatic Cancer, best j: 0.70, global p-value: 1.658e-232
Cancer type: Prostate Cancer, best j: 0.70, global p-value: 1.018e-121
